In [33]:
import sys
sys.path.append("../src")
import os

import pandas as pd
import numpy as np
from etf_transformations import *
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import randint, uniform

In [34]:
df = pd.read_csv('../data/preprocessed/agg_emb/XLF_v4.csv')

df = sign_next_day(df)
df = drop_sign_and_return(df)

df.head()

,Date,Price,avg_positive_XLF,avg_neutral_XLF,avg_negative_XLF,n_XLF,sent_index_XLF,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,24.879965,0.061260,0.535400,0.403340,4.0,-0.342080,-0.003630,-0.025403,-0.005257,...,-0.041143,0.014036,-0.033065,-0.033968,0.009230,-0.046950,0.004274,0.024519,1.0,-1.0
1,2018-03-21,24.871304,0.263512,0.485453,0.251035,14.0,0.012478,0.001563,-0.050504,-0.019049,...,-0.059143,0.007399,-0.055302,-0.039515,-0.000138,-0.066466,0.005075,-0.024416,1.0,-1.0
2,2018-03-22,23.953358,0.281100,0.323045,0.395855,11.0,-0.114755,0.002994,-0.036866,-0.010311,...,-0.056243,0.016681,-0.040932,-0.007561,0.011109,-0.076687,0.010291,-0.010929,1.0,-1.0
3,2018-03-23,23.225918,0.288174,0.307246,0.404580,6.0,-0.116405,-0.012174,-0.035777,-0.016196,...,-0.033069,0.029601,-0.046962,-0.005147,0.038169,-0.097698,0.013953,-0.036602,1.0,1.0
4,2018-03-26,23.979336,0.220507,0.469852,0.309642,11.0,-0.089135,-0.005624,-0.024752,-0.004313,...,-0.031487,0.017077,-0.037181,-0.028825,0.034319,-0.078715,0.038083,-0.010276,1.0,-1.0


In [35]:
df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()
df = df.sort_values("Date").reset_index(drop=True)

df["Return_next_day"] = np.log(df["Price"].shift(-1) / df["Price"])
df["AbsReturn_next_day"] = df["Return_next_day"].abs()

df["no_news"] = df["n_XLF"].isna().astype(int) # a day where there were no sector specific news
df.head(10)


,Date,Price,avg_positive_XLF,avg_neutral_XLF,avg_negative_XLF,n_XLF,sent_index_XLF,emb_0,emb_1,emb_2,...,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day,Return_next_day,AbsReturn_next_day,no_news
0,2018-03-20,24.879965,0.061260,0.535400,0.403340,4.0,-0.342080,-0.003630,-0.025403,-0.005257,...,-0.033968,0.009230,-0.046950,0.004274,0.024519,1.0,-1.0,-0.000348,0.000348,0
1,2018-03-21,24.871304,0.263512,0.485453,0.251035,14.0,0.012478,0.001563,-0.050504,-0.019049,...,-0.039515,-0.000138,-0.066466,0.005075,-0.024416,1.0,-1.0,-0.037606,0.037606,0
2,2018-03-22,23.953358,0.281100,0.323045,0.395855,11.0,-0.114755,0.002994,-0.036866,-0.010311,...,-0.007561,0.011109,-0.076687,0.010291,-0.010929,1.0,-1.0,-0.030840,0.030840,0
3,2018-03-23,23.225918,0.288174,0.307246,0.404580,6.0,-0.116405,-0.012174,-0.035777,-0.016196,...,-0.005147,0.038169,-0.097698,0.013953,-0.036602,1.0,1.0,0.031924,0.031924,0
4,2018-03-26,23.979336,0.220507,0.469852,0.309642,11.0,-0.089135,-0.005624,-0.024752,-0.004313,...,-0.028825,0.034319,-0.078715,0.038083,-0.010276,1.0,-1.0,-0.019694,0.019694,0
5,2018-03-27,23.511696,0.208302,0.501280,0.290419,8.0,-0.082117,0.008858,-0.044920,-0.005257,...,-0.046846,0.031692,-0.089109,0.010091,0.004574,1.0,1.0,0.001840,0.001840,0
6,2018-03-28,23.554998,0.092945,0.652902,0.254153,8.0,-0.161208,0.003700,-0.017640,0.008789,...,-0.029270,0.003462,-0.093493,-0.006516,0.024834,1.0,1.0,0.013511,0.013511,0
7,2018-03-29,23.875416,0.055242,0.592329,0.352428,5.0,-0.297186,0.000013,-0.013250,-0.006634,...,-0.006435,-0.023151,-0.084078,0.012592,0.008806,1.0,-1.0,-0.022374,0.022374,0
8,2018-04-02,23.347151,0.046485,0.476633,0.476883,6.0,-0.430398,-0.013063,-0.034718,-0.003257,...,0.019447,0.047689,-0.111716,0.016507,-0.010199,1.0,1.0,0.013631,0.013631,0
9,2018-04-03,23.667574,0.627429,0.324922,0.047649,3.0,0.579779,0.022106,-0.012358,0.013611,...,0.030860,0.067125,-0.091592,-0.007063,0.020872,1.0,1.0,0.010555,0.010555,0


In [36]:
q = 0.70  # try 0.70, 0.75, 0.80
tau = df["AbsReturn_next_day"].quantile(q)
df["HighVol_next_day"] = (df["AbsReturn_next_day"] > tau).astype(int)


df["abs_ret_t"] = np.log(df["Price"] / df["Price"].shift(1)).abs()
df["vol_5"] = df["abs_ret_t"].rolling(5).mean()


df = df.dropna(subset=["HighVol_next_day"])
df.head(10)

,Date,Price,avg_positive_XLF,avg_neutral_XLF,avg_negative_XLF,n_XLF,sent_index_XLF,emb_0,emb_1,emb_2,...,emb_382,emb_383,is_trading_day,Sign_next_day,Return_next_day,AbsReturn_next_day,no_news,HighVol_next_day,abs_ret_t,vol_5
0,2018-03-20,24.879965,0.061260,0.535400,0.403340,4.0,-0.342080,-0.003630,-0.025403,-0.005257,...,0.004274,0.024519,1.0,-1.0,-0.000348,0.000348,0,0,NaN,NaN
1,2018-03-21,24.871304,0.263512,0.485453,0.251035,14.0,0.012478,0.001563,-0.050504,-0.019049,...,0.005075,-0.024416,1.0,-1.0,-0.037606,0.037606,0,1,0.000348,NaN
2,2018-03-22,23.953358,0.281100,0.323045,0.395855,11.0,-0.114755,0.002994,-0.036866,-0.010311,...,0.010291,-0.010929,1.0,-1.0,-0.030840,0.030840,0,1,0.037606,NaN
3,2018-03-23,23.225918,0.288174,0.307246,0.404580,6.0,-0.116405,-0.012174,-0.035777,-0.016196,...,0.013953,-0.036602,1.0,1.0,0.031924,0.031924,0,1,0.030840,NaN
4,2018-03-26,23.979336,0.220507,0.469852,0.309642,11.0,-0.089135,-0.005624,-0.024752,-0.004313,...,0.038083,-0.010276,1.0,-1.0,-0.019694,0.019694,0,1,0.031924,NaN
5,2018-03-27,23.511696,0.208302,0.501280,0.290419,8.0,-0.082117,0.008858,-0.044920,-0.005257,...,0.010091,0.004574,1.0,1.0,0.001840,0.001840,0,0,0.019694,0.024082
6,2018-03-28,23.554998,0.092945,0.652902,0.254153,8.0,-0.161208,0.003700,-0.017640,0.008789,...,-0.006516,0.024834,1.0,1.0,0.013511,0.013511,0,1,0.001840,0.024381
7,2018-03-29,23.875416,0.055242,0.592329,0.352428,5.0,-0.297186,0.000013,-0.013250,-0.006634,...,0.012592,0.008806,1.0,-1.0,-0.022374,0.022374,0,1,0.013511,0.019562
8,2018-04-02,23.347151,0.046485,0.476633,0.476883,6.0,-0.430398,-0.013063,-0.034718,-0.003257,...,0.016507,-0.010199,1.0,1.0,0.013631,0.013631,0,1,0.022374,0.017869
9,2018-04-03,23.667574,0.627429,0.324922,0.047649,3.0,0.579779,0.022106,-0.012358,0.013611,...,-0.007063,0.020872,1.0,1.0,0.010555,0.010555,0,0,0.013631,0.014210


In [37]:
df.isna().sum()

Date                  0
Price                 0
avg_positive_XLF      5
avg_neutral_XLF       5
avg_negative_XLF      5
                     ..
AbsReturn_next_day    1
no_news               0
HighVol_next_day      0
abs_ret_t             1
vol_5                 5
Length: 399, dtype: int64

In [38]:
rows_with_nan = df[df.isna().any(axis=1)]
rows_with_nan


,Date,Price,avg_positive_XLF,avg_neutral_XLF,avg_negative_XLF,n_XLF,sent_index_XLF,emb_0,emb_1,emb_2,...,emb_382,emb_383,is_trading_day,Sign_next_day,Return_next_day,AbsReturn_next_day,no_news,HighVol_next_day,abs_ret_t,vol_5
0,2018-03-20,24.879965,0.061260,0.535400,0.403340,4.0,-0.342080,-0.003630,-0.025403,-0.005257,...,0.004274,0.024519,1.0,-1.0,-0.000348,0.000348,0,0,NaN,NaN
1,2018-03-21,24.871304,0.263512,0.485453,0.251035,14.0,0.012478,0.001563,-0.050504,-0.019049,...,0.005075,-0.024416,1.0,-1.0,-0.037606,0.037606,0,1,0.000348,NaN
2,2018-03-22,23.953358,0.281100,0.323045,0.395855,11.0,-0.114755,0.002994,-0.036866,-0.010311,...,0.010291,-0.010929,1.0,-1.0,-0.030840,0.030840,0,1,0.037606,NaN
3,2018-03-23,23.225918,0.288174,0.307246,0.404580,6.0,-0.116405,-0.012174,-0.035777,-0.016196,...,0.013953,-0.036602,1.0,1.0,0.031924,0.031924,0,1,0.030840,NaN
4,2018-03-26,23.979336,0.220507,0.469852,0.309642,11.0,-0.089135,-0.005624,-0.024752,-0.004313,...,0.038083,-0.010276,1.0,-1.0,-0.019694,0.019694,0,1,0.031924,NaN
73,2018-07-03,23.030825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.002263,0.002263,1,0,0.009022,0.007006
297,2019-05-24,23.731804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-1.0,-0.011608,0.011608,1,0,0.007849,0.007875
349,2019-08-08,24.207939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-1.0,-0.003307,0.003307,1,0,0.018886,0.016501
392,2019-10-09,24.126066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.010305,0.010305,1,0,0.009292,0.011366
447,2019-12-27,27.604074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-1.0,-0.002929,0.002929,1,0,0.002596,0.002773


In [39]:
df = df.dropna()
df.isna().sum()

Date                  0
Price                 0
avg_positive_XLF      0
avg_neutral_XLF       0
avg_negative_XLF      0
                     ..
AbsReturn_next_day    0
no_news               0
HighVol_next_day      0
abs_ret_t             0
vol_5                 0
Length: 399, dtype: int64

In [40]:
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import randint, uniform

drop_cols = ["Date", "HighVol_next_day", "Return_next_day", "AbsReturn_next_day", "Return", "Sign"]
X = df.drop(columns=[c for c in drop_cols if c in df.columns])
y = df["HighVol_next_day"]

catboost = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=12,
    verbose=0,
    allow_writing_files=False
)

param_distributions = {
    "depth": randint(3, 7),
    "learning_rate": uniform(0.01, 0.19),
    "iterations": randint(100, 500),
    "l2_leaf_reg": uniform(1, 14),
    "border_count": randint(32, 128),
    "random_strength": uniform(0.5, 5),
    "min_data_in_leaf": randint(5, 30),
    "rsm": uniform(0.6, 0.4),
    "boosting_type": ["Ordered", "Plain"],
    "bootstrap_type": ["Bernoulli"],# , "MVS"],
    "subsample": uniform(0.6, 0.4),
}

tscv = TimeSeriesSplit(n_splits=5)

random_search = RandomizedSearchCV(
    estimator=catboost,
    param_distributions=param_distributions,
    n_iter=40,
    cv=tscv,
    scoring="roc_auc",
    random_state=12,
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)

random_search.fit(X, y)

print(f"Best AUC: {random_search.best_score_:.4f}")
print("Best parameters:")
for p, v in random_search.best_params_.items():
    print(f"  {p}: {v}")


Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best AUC: 0.5921
Best parameters:
  boosting_type: Ordered
  bootstrap_type: Bernoulli
  border_count: 47
  depth: 5
  iterations: 450
  l2_leaf_reg: 13.778975918391541
  learning_rate: 0.042280669577058255
  min_data_in_leaf: 10
  random_strength: 3.714666368133583
  rsm: 0.9651383861629813
  subsample: 0.8400871054647666


In [41]:
q = 0.80  # try 0.70, 0.75, 0.80
tau = df["AbsReturn_next_day"].quantile(q)
df["HighVol_next_day"] = (df["AbsReturn_next_day"] > tau).astype(int)


df["abs_ret_t"] = np.log(df["Price"] / df["Price"].shift(1)).abs()
df["vol_5"] = df["abs_ret_t"].rolling(5).mean()


df = df.dropna(subset=["HighVol_next_day"])
df.head(10)

,Date,Price,avg_positive_XLF,avg_neutral_XLF,avg_negative_XLF,n_XLF,sent_index_XLF,emb_0,emb_1,emb_2,...,emb_382,emb_383,is_trading_day,Sign_next_day,Return_next_day,AbsReturn_next_day,no_news,HighVol_next_day,abs_ret_t,vol_5
5,2018-03-27,23.511696,0.208302,0.501280,0.290419,8.0,-0.082117,0.008858,-0.044920,-0.005257,...,0.010091,0.004574,1.0,1.0,0.001840,0.001840,0,0,NaN,NaN
6,2018-03-28,23.554998,0.092945,0.652902,0.254153,8.0,-0.161208,0.003700,-0.017640,0.008789,...,-0.006516,0.024834,1.0,1.0,0.013511,0.013511,0,0,0.001840,NaN
7,2018-03-29,23.875416,0.055242,0.592329,0.352428,5.0,-0.297186,0.000013,-0.013250,-0.006634,...,0.012592,0.008806,1.0,-1.0,-0.022374,0.022374,0,1,0.013511,NaN
8,2018-04-02,23.347151,0.046485,0.476633,0.476883,6.0,-0.430398,-0.013063,-0.034718,-0.003257,...,0.016507,-0.010199,1.0,1.0,0.013631,0.013631,0,0,0.022374,NaN
9,2018-04-03,23.667574,0.627429,0.324922,0.047649,3.0,0.579779,0.022106,-0.012358,0.013611,...,-0.007063,0.020872,1.0,1.0,0.010555,0.010555,0,0,0.013631,NaN
10,2018-04-04,23.918718,0.050521,0.906664,0.042815,2.0,0.007707,-0.049135,-0.054272,-0.049306,...,0.025393,-0.024925,1.0,1.0,0.007934,0.007934,0,0,0.010555,0.012382
11,2018-04-05,24.109234,0.273679,0.516308,0.210013,7.0,0.063665,-0.016982,0.025092,-0.005531,...,0.034368,-0.010119,1.0,-1.0,-0.024360,0.024360,0,1,0.007934,0.013601
12,2018-04-06,23.529022,0.333590,0.415331,0.251079,8.0,0.082511,-0.020975,-0.025775,-0.001227,...,0.018157,0.002133,1.0,1.0,0.005506,0.005506,0,0,0.024360,0.015771
13,2018-04-09,23.658920,0.094780,0.605769,0.299451,14.0,-0.204671,-0.001426,-0.021839,-0.016479,...,0.016951,0.010751,1.0,1.0,0.014896,0.014896,0,0,0.005506,0.012397
14,2018-04-10,24.013977,0.214749,0.428031,0.357220,6.0,-0.142471,-0.017991,-0.034595,-0.009132,...,0.056691,-0.015679,1.0,-1.0,-0.011972,0.011972,0,0,0.014896,0.012650


In [42]:
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import randint, uniform

drop_cols = ["Date", "HighVol_next_day", "Return_next_day", "AbsReturn_next_day", "Return", "Sign"]
X = df.drop(columns=[c for c in drop_cols if c in df.columns])
y = df["HighVol_next_day"]

catboost = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=12,
    verbose=0,
    allow_writing_files=False
)

param_distributions = {
    "depth": randint(3, 7),
    "learning_rate": uniform(0.01, 0.19),
    "iterations": randint(100, 500),
    "l2_leaf_reg": uniform(1, 14),
    "border_count": randint(32, 128),
    "random_strength": uniform(0.5, 5),
    "min_data_in_leaf": randint(5, 30),
    "rsm": uniform(0.6, 0.4),
    "boosting_type": ["Ordered", "Plain"],
    "bootstrap_type": ["Bernoulli"],# , "MVS"],
    "subsample": uniform(0.6, 0.4),
}

tscv = TimeSeriesSplit(n_splits=5)

random_search = RandomizedSearchCV(
    estimator=catboost,
    param_distributions=param_distributions,
    n_iter=40,
    cv=tscv,
    scoring="roc_auc",
    random_state=12,
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)

random_search.fit(X, y)

print(f"Best AUC: {random_search.best_score_:.4f}")
print("Best parameters:")
for p, v in random_search.best_params_.items():
    print(f"  {p}: {v}")


Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best AUC: 0.6077
Best parameters:
  boosting_type: Plain
  bootstrap_type: Bernoulli
  border_count: 65
  depth: 5
  iterations: 219
  l2_leaf_reg: 7.795283791277936
  learning_rate: 0.15594548927224025
  min_data_in_leaf: 23
  random_strength: 2.780254616013186
  rsm: 0.8252564550066286
  subsample: 0.9205061529136007
